In [6]:


import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from torch.optim import Adam
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda
from torch.utils.data import DataLoader
from collections import defaultdict
import pandas as pd
from random import choice
import os
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
import io
import sys
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torchvision
from torchvision.datasets import MNIST


def F_Mnist(batch_size_train,batch_size_test):
  compress_factor = 1
  reshape_f = lambda x: torch.reshape(x[0, ::compress_factor, ::compress_factor], (-1, ))

  transforms_train = Compose([
    #transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
    #transforms.RandomRotation(5),     #Rotates the image to a specified angel
    transforms.RandomAffine(0, translate= (.025,.025),shear=0), #Performs actions like zooms, change shear angles.
    #transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1), # Set the color params
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),Lambda(lambda x: torch.flatten(x)) ])
  transforms_val = transforms.Compose([transforms.ToTensor(),  transforms.Normalize((0.1307,), (0.3081,)),Lambda(lambda x: torch.flatten(x))])

  train_dataset = datasets.FashionMNIST("~/.pytorch/F_MNIST_data/", train=True, download=True, transform=transforms_train)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)

  val_dataset = datasets.FashionMNIST("~/.pytorch/F_MNIST_data/", train=False, download=True, transform=transforms_val)
  val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size_test, shuffle=False)
  return train_loader,val_loader


def MNIST_loaders(train_batch_size=100, test_batch_size=10000):

    transform = Compose([
        ToTensor(),
        Normalize((0.1307,), (0.3081,)),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = DataLoader(
        MNIST('./data/', train=True,
              download=True,
              transform=transform),
        batch_size=train_batch_size, shuffle=True)

    test_loader = DataLoader(
        MNIST('./data/', train=False,
              download=True,
              transform=transform),
        batch_size=test_batch_size, shuffle=False)

    return train_loader, test_loader


def overlay_y_on_x(x, y):
    x_ = x.clone()
    x_[:,:10] *= 0.0
    x_[range(x.shape[0]), y] = x.max()
    return x_



class CustomCosineSimilarity(nn.Module):
    def __init__(self):
        super(CustomCosineSimilarity, self).__init__()

    def forward(self, x, y):
        # 确保x和y扩充到2D
        if x.dim() == 1:
            x = x.unsqueeze(0)
        if y.dim() == 1:
            y = y.unsqueeze(0)
        
        # 计算内积，考虑到我们处理的是2D张量，这里使用matmul而非dot！！
        inner_product = torch.matmul(x, y.T)

        # 用最大值替代模长，避免求导的复杂性
        max_x = torch.max(x, dim=1, keepdim=True)[0]
        max_y = torch.max(y, dim=1, keepdim=True)[0]

        result = inner_product / (max_x * max_y.T + 1e-6)
        return result



class Layer(nn.Linear):
    def __init__(self, in_features, out_features,
                 bias=True, device=None, dtype=None):
        super().__init__(in_features, out_features, bias, device, dtype)
        self.P=nn.Linear(in_features, out_features,bias=True).cuda()
        self.opt = Adam(self.P.parameters(),weight_decay=0, lr=0.0001)
        self.threshold = 4
        self.running_loss=0.0
        self.num_epochs_internal =25
        self.relu = torch.nn.ReLU()
        self.gelu=torch.nn.GELU()


    def forward(self, x,k):
      x=self.P(x)
      return self.gelu(x)



    def goodness(self,x_pos,x_neg,k):
      # cos = nn.CosineSimilarity(dim=1, eps=1e-6)
      cos = CustomCosineSimilarity()
      if k==0:
        g_p = (self.forward(x_pos,k))
        g_pos =cos(g_p, p_vector0)

        g_n = (self.forward(x_neg,k))
        g_neg =cos(g_n, p_vector0)
      if k==1:
        g_p = (self.forward(x_pos,k))
        g_pos =cos(g_p, p_vector1)

        g_n = (self.forward(x_neg,k))
        g_neg =cos(g_n, p_vector1)

      return g_pos,g_neg


    def train(self, x_pos, x_neg,k):
        self.running_loss=0.0
        for i in range(self.num_epochs_internal):

          g_pos,g_neg=self.goodness(x_pos,x_neg,k)
          delta=g_pos-g_neg
          loss = (torch.log(1 + torch.exp(
              -self.threshold*delta ))).mean()
          self.opt.zero_grad()
          loss.backward()
          self.opt.step()
          self.running_loss+=loss.item()
        return self.forward(x_pos,k).detach(), self.forward(x_neg,k).detach(), self.running_loss/self.num_epochs_internal


acc_test=[]
acc_train=[]

class FNet(torch.nn.Module):
    def __init__(self, dims):
        super().__init__()
        self.J=torch.ones(40,40).cuda()
        self.e=torch.ones(1,40).cuda()
        self.layers = []
        self.para_weight=[]
        self.para_bias=[]
        self.loss_save=defaultdict(list)
        self.param_save=[]
        self.num_epochs =20
        self.num_label=10
        self.POOL = nn.AvgPool2d(3, stride=2,padding=1)
        self.Pad = nn.ZeroPad2d((0,0,0,4))
        self.sigmoid=nn.Sigmoid()
        self.N_layer=int(len(dims)/2)
        self.loss_list=[0] * int(len(dims)/2)
        for d in range(0, (len(dims)), 2):
            self.layers += [Layer( dims[d ], dims[d+1 ]).cuda()]

    def predict(self, x):
        goodness_per_label = []
        for label in range(self.num_label):
            h = overlay_y_on_x(x, label)
            goodness = []
            cos = nn.CosineSimilarity(dim=1, eps=1e-6)
            # cos = CustomCosineSimilarity()
            for k, layer in enumerate(self.layers):
              h = layer(h,k)
              g = (h)
              if k==0:
                goodness += [cos(g, p_vector0)]
              if k==1:
                goodness += [cos(g, p_vector1)]

            goodness_per_label += [sum(goodness).unsqueeze(1)]
        goodness_per_label = torch.cat(goodness_per_label, 1)
        return goodness_per_label.argmax(1)

    def train(self, train_loader,val_loader):
        k=0
        for i in range(self.num_epochs):
            self.loss_list=[0] * self.N_layer
            for j, data in enumerate(train_loader, 0):
              x, y = data
              x, y = x.cuda(),y.cuda()
              x_pos = overlay_y_on_x(x, y)
              y_n=y.clone()
              for s in range(x.size(0)):
                y_n[s]=int(choice(list(set(range(0, self.num_label)) - set([y[s].item()]))))
              x_neg = overlay_y_on_x(x, y_n)
              h_pos, h_neg = x_pos.cuda(), x_neg.cuda()

              for k, layer in enumerate(self.layers):

                h_pos, h_neg,loss = layer.train(h_pos,  h_neg,k)

                self.loss_list[k]=self.loss_list[k]+loss


            self.loss_list=np.divide(self.loss_list, j+1)


            for l in range(self.N_layer):
                  self.loss_save[l].append(self.loss_list[l])
            print( f'[Epochs: {i + 1}], Loss: {self.loss_list}' )
            # if i%10==9:
            with torch.no_grad():
              er=0.0
              for g, data in enumerate(train_loader, 0):
                x_t, y_t = data
                x_t, y_t = x_t.cuda(), y_t.cuda()
                er+=( self.predict(x_t).eq(y_t).float().mean().item())

              print('Train Accuracy:', (er)/(g+1)*100)
              acc_train.append((er)/(g+1)*100)
              er=0.0
              for t, data in enumerate(val_loader, 0):
                x_te, y_te = data
                x_te, y_te = x_te.cuda(), y_te.cuda()
                er+=( self.predict(x_te).eq(y_te).float().mean().item())
              print('Test Accuracy:', (er)/(t+1)*100)
              acc_test.append((er)/(t+1)*100)



p_vector0=torch.normal(0, 1, size=(1, 1000))
p_vector0=p_vector0/ (p_vector0.norm(2, 1, keepdim=True) + 1e-4)
p_vector1=torch.normal(0, 1, size=(1, 1000))
p_vector1=p_vector1/ (p_vector1.norm(2, 1, keepdim=True) + 1e-4)

p_vector0=(p_vector0.repeat(10000, 1)).cuda()
p_vector1=(p_vector1.repeat(10000, 1)).cuda()



torch.set_default_dtype(torch.float32)
train_loader,val_loader=MNIST_loaders(10000,10000)
net = FNet([28*28,1000,1000,1000])
net.train(train_loader,val_loader)



[Epochs: 1], Loss: [0.13042461 0.07806853]
Train Accuracy: 90.28666416803995
Test Accuracy: 90.62999486923218
[Epochs: 2], Loss: [0.03802382 0.02492961]
Train Accuracy: 91.96166396141052
Test Accuracy: 91.97999835014343
[Epochs: 3], Loss: [0.02603848 0.01683783]
Train Accuracy: 93.10999810695648
Test Accuracy: 92.91999936103821
[Epochs: 4], Loss: [0.01848117 0.0123628 ]
Train Accuracy: 94.09499764442444
Test Accuracy: 93.61000061035156
[Epochs: 5], Loss: [nan nan]
Train Accuracy: 9.871666505932808
Test Accuracy: 9.799999743700027
[Epochs: 6], Loss: [nan nan]
Train Accuracy: 9.871666381756466
Test Accuracy: 9.799999743700027
[Epochs: 7], Loss: [nan nan]
Train Accuracy: 9.871666381756466
Test Accuracy: 9.799999743700027


KeyboardInterrupt: 

In [5]:
p_vector0=torch.normal(0, 1, size=(1, 1000))
p_vector0=p_vector0/ (p_vector0.norm(2, 1, keepdim=True) + 1e-4)
p_vector1=torch.normal(0, 1, size=(1, 1000))
p_vector1=p_vector1/ (p_vector1.norm(2, 1, keepdim=True) + 1e-4)

p_vector0=(p_vector0.repeat(10000, 1)).cuda()
p_vector1=(p_vector1.repeat(10000, 1)).cuda()


In [8]:
p_vector0.shape

torch.Size([10000, 1000])

In [10]:
torch.normal(0, 1, size=(1, 1000)), torch.normal(0, 1, size=(1, 1000)).shape

(tensor([[ 4.5906e-01, -1.2064e+00,  3.5593e-01,  8.9577e-01,  1.2560e+00,
           1.4983e+00,  2.3623e-02, -1.9295e-01,  5.7306e-03,  7.7744e-01,
           6.2530e-01, -1.6878e+00,  8.9308e-01, -1.6010e+00,  1.8778e+00,
          -2.7001e-01,  2.2697e-01, -5.8961e-01,  1.4445e+00, -2.7240e-01,
          -1.6261e+00, -1.2220e+00,  1.0766e-01, -8.3358e-01, -1.0103e+00,
          -1.6387e+00,  1.6424e+00,  1.2925e+00,  7.4663e-01, -1.6416e-02,
          -1.9127e-01, -1.8558e+00,  1.8371e-01,  6.2509e-01,  2.2333e-01,
          -1.1747e+00, -2.1887e+00, -1.5026e-01,  1.3098e+00, -5.3937e-01,
          -1.0065e+00,  4.8031e-01,  3.8939e-01,  1.8480e+00,  8.4694e-01,
           1.1861e+00, -1.3060e-01,  1.4472e+00,  5.3964e-01,  3.8300e-01,
          -7.0488e-01, -4.2418e-01,  4.8299e-01,  8.8429e-01,  4.0133e-01,
           2.1059e-01, -6.9364e-01,  3.0224e-02,  3.3514e-01,  9.0764e-02,
           5.6317e-01, -7.1562e-01,  8.3180e-01, -1.6389e+00,  7.9661e-01,
          -3.5473e-02,  7